In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tables
from random import shuffle
from IPython.display import clear_output
from sklearn import metrics
from tqdm import tqdm
import time
import seaborn as sns
%matplotlib inline

/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import os
USE_GPU = True
os.environ['CUDA_VISIBLE_DEVICES'] = '3' if USE_GPU else ''
EPS = 1e-15

In [3]:
lpmt_hits = pd.read_hdf('../data/lpmt_hits.h5', mode='r') 
spmt_hits = pd.read_hdf('../data/spmt_hits.h5', mode='r')
lpmt_pos = pd.read_csv('data/lpmt_pos.csv') 
spmt_pos = pd.read_csv('data/spmt_pos.csv') 
true_info = pd.read_csv('data/true_info.csv') 

In [4]:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

24243798016


In [5]:
len(lpmt_hits), len(spmt_hits)

(929768817, 33958478)

In [6]:
lpmt_hits.head() # spmt_hits.head() 

,event,hitTime,isDN,pmtID
0,0,249.992615,False,14175
1,0,40.010311,False,17319
2,0,162.123199,False,16882
3,0,51.875614,False,14951
4,0,79.817497,False,10947


In [7]:
lpmt_pos.head() # spmt_pos.head()

,pmt_id,pmt_x,pmt_y,pmt_z
0,0,1.049021e+03,0.0000,19171.32
1,1,9.084785e+02,524.5103,19171.32
2,2,5.245103e+02,908.4785,19171.32
3,3,6.423399e-14,1049.0206,19171.32
4,4,-5.245103e+02,908.4785,19171.32


In [8]:
true_info.head()

,E,R,evtID,x,y,z
0,4.747791,14610.378,0,8290.7790,11995.6180,911.74286
1,3.919721,14630.141,1,11397.6320,5407.4497,-7409.08200
2,6.823932,14573.132,2,14063.3380,-3812.8540,246.65280
3,3.765940,16820.080,3,-2377.9307,-16317.7020,3315.59030
4,3.217473,13026.938,4,-8617.1170,868.5116,9730.98600


# KNN

In [9]:
total_pos = lpmt_pos.append(spmt_pos) 

In [10]:
neibours = np.zeros((len(total_pos), 60), dtype='int32')

coord_list = []
for _, x, y, z in total_pos.itertuples(index=False):
    coord_list.append((x, y, z))

coord = np.array(coord_list)

for i, (id1, x1, y1, z1) in tqdm(enumerate(total_pos.itertuples(index=False)), total=len(total_pos)):
    dists = np.sum(np.square(coord - (x1, y1, z1)), axis=1)
    neibours[i] = np.argsort(dists)[1:101]

100%|██████████| 42691/42691 [02:35<00:00, 274.72it/s]


In [11]:
pmt_id_map = {v:i for i, v in enumerate(total_pos['pmt_id'].values)}

In [12]:
lpmt_n_hits = pd.read_csv('data/lpmt_n_hits.csv', header=None).values
spmt_n_hits = pd.read_csv('data/spmt_n_hits.csv', header=None).values

In [13]:
len(lpmt_n_hits)

100000

In [14]:
max_events = 100 

In [15]:
def generate_hit_feature(f):
    lpmt_result = np.zeros(len(lpmt_hits))
    spmt_result = np.zeros(len(spmt_hits))
    total_lpmt_hits = 0
    total_spmt_hits = 0

    for event_id in tqdm(range(max_events)):
        lnh = int(lpmt_n_hits[event_id])
        snh = int(spmt_n_hits[event_id])
    
        event_hits = lpmt_hits[total_lpmt_hits:total_lpmt_hits+lnh].append(
                     spmt_hits[total_spmt_hits:total_spmt_hits+snh])
        
        values = np.array(f(event_hits))
        
        lpmt_result[total_lpmt_hits:total_lpmt_hits+lnh] = values[:lnh]
        spmt_result[total_spmt_hits:total_spmt_hits+snh] = values[lnh:]
    
        total_lpmt_hits += lnh
        total_spmt_hits += snh
    
    return lpmt_result, spmt_result

In [16]:
def count_neibours(top_n, dtime):
    def inner(event_hits):
        d = {}
        for _, time, _, mtID in event_hits.itertuples(index=False):
            if (mtID in pmt_id_map):
                i = pmt_id_map[mtID]
                l = d.get(i, [])
                l.append(time)
                d[i] = l

        d = {k: np.array(v) for k, v in d.items()}

        values = []
        for _, time, _, mtID in event_hits.itertuples(index=False):
            if (mtID not in pmt_id_map):
                values.append(0)
                continue
            i = pmt_id_map[mtID]
            value = 0 
            for j in neibours[i,0:top_n]:
                if j in d:
                    value += np.sum(np.abs(time - d[j]) < dtime)
            values.append(value)
        
        return values
    return inner

In [17]:
def count_hits_in_event(event_hits):
    d = {}
    for _, time, _, mtID in event_hits.itertuples(index=False):
        d[mtID] = d.get(mtID, 0) + 1
        
    values = []
    for _, time, _, mtID in event_hits.itertuples(index=False):
        values.append(d[mtID])
            
    return values

In [18]:
lpmt_f1, spmt_f1 = generate_hit_feature(count_hits_in_event)

100%|██████████| 100/100 [00:02<00:00, 43.94it/s]


In [19]:
lpmt_f2, spmt_f2 = generate_hit_feature(count_neibours(20, 100))

100%|██████████| 100/100 [01:20<00:00,  1.24it/s]


In [20]:
lpmt_f3, spmt_f3 = generate_hit_feature(count_neibours(60, 100))

100%|██████████| 100/100 [04:25<00:00,  2.65s/it]


In [21]:
X_lpmt = np.stack([lpmt_f1, lpmt_f2, lpmt_f3], axis=1)
y_lpmt = lpmt_hits['isDN'].values

In [22]:
X_lpmt = X_lpmt[lpmt_hits['event'] < max_events]
y_lpmt = y_lpmt[lpmt_hits['event'] < max_events]

In [23]:
X_spmt = np.stack([spmt_f1, spmt_f2, spmt_f3], axis=1)
y_spmt = spmt_hits['isDN'].values

In [24]:
X_spmt = X_spmt[spmt_hits['event'] < max_events]
y_spmt = y_spmt[spmt_hits['event'] < max_events]

In [26]:
X_lpmt.shape

(954217, 3)

# XGBoost

In [48]:
import xgboost as xgb

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_lpmt, y_lpmt)

In [49]:
from sklearn.metrics import accuracy_score

In [50]:
np.mean(X_lpmt[y_lpmt == 0, 2]), np.mean(X_lpmt[y_lpmt == 1, 2])

(96.24546458933484, 2.7082517406440383)

In [51]:
accuracy_score(y_lpmt, np.zeros(len(y_lpmt)))

0.922935768279123

In [52]:
y_pred = gbm.predict(X_lpmt)
accuracy_score(y_lpmt, y_pred)

/opt/conda/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9446195152674915

In [53]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_spmt, y_spmt)

In [54]:
y_pred = gbm.predict(X_spmt)
accuracy_score(y_spmt, y_pred)

/opt/conda/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9426564569059819